In [2]:
import scanpy as sc
import pandas as pd 
import numpy as np
import seaborn as sns

from scipy.stats import pearsonr

from sklearn.linear_model import ElasticNetCV, RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import make_scorer, r2_score

In [3]:
import sys
import os
from importlib import reload

# Add the path of your package
package_path = os.path.abspath("/Genomics/pritykinlab/tamjeed/github_packages/GlossPath/")
sys.path.insert(0, package_path)

In [4]:
import matplotlib.pyplot as plt

In [5]:
import pickle

In [60]:
og_adata = sc.read_h5ad('../joint_notebooks/datasets/lipstic_tumor_data_aug_11_2024.h5ad')

In [7]:
from scipy.sparse import csr_matrix

In [8]:
with open('../pathway_computations/cd40neighbors.pickle', 'rb') as handle:
    cd40neighbors = pickle.load(handle)

In [70]:
with open('../pathway_computations/cd40_filtered.pickle', 'rb') as handle:
    cd40_filtered_neighbors = pickle.load(handle)

In [117]:
cd40_subset_adata = og_adata[:, og_adata.var.index.isin(cd40_filtered_neighbors['cd40_f_in'])]
non_cd40_subset_adata = og_adata[:, ~og_adata.var.index.isin(cd40_filtered_neighbors['cd40_f_in'])]
intervals = [(1e0, 1e1), (1e1, 1e2), (1e2, 1e3), (1e3, 1e4), (1e4, 1e5), (1e5, 1e6)]
y = og_adata.obs['biotin_raw']
perturbed_genes = {}
for i in range(10):
    print(i)
    adata = og_adata.copy()
    perturbed_genes[i] = {}
    for interval in intervals:
        int_non_cd40_subset_adata = non_cd40_subset_adata[:, non_cd40_subset_adata.var['n_counts'].between(interval[0], interval[1])].copy()
        int_cd40_subset_adata = cd40_subset_adata[:, cd40_subset_adata.var['n_counts'].between(interval[0], interval[1])].copy()
        
        total_non_cd40_genes = int_non_cd40_subset_adata.shape[1]
        target_gene_count = int_cd40_subset_adata.shape[1]

        np.random.seed(i)
        # Randomly sample 173 gene indices from the total 318 genes
        selected_genes = np.random.choice(total_non_cd40_genes, target_gene_count, replace=False)
        print(selected_genes)

        # Subset the AnnData object to the selected genes
        int_non_cd40_subset_adata = int_non_cd40_subset_adata[:, selected_genes].copy()
        perturbed_genes[i][interval] = int_non_cd40_subset_adata.var.index

        perturb_df = int_non_cd40_subset_adata.to_df().copy()

        np.random.seed(i)
        random_factors = np.random.uniform(0.5, 1.5, size=(perturb_df.shape[0], 1))
        random_factors.flatten()
        result_df = np.round(perturb_df * random_factors).astype(int).astype(float)
        adata_df[result_df.columns] = result_df
        
    result_y = np.round(y * random_factors.flatten()).astype(int).astype(float)
    result_y = np.where(y >= 1, np.maximum(result_y, 1), result_y)
    
    print(np.percentile(result_y, 5))
    adata.obs['biotin_raw_perturbed'] = result_y
    csr_matrix(adata_df)
    adata.X = csr_matrix(adata_df)

    adata.write('../joint_notebooks/datasets/{}_lipstic_tumor_data_perturbed_filtered_binned_v3_oct_25_2024.h5ad'.format(i)) 
    

0
[]
[]
[ 989  463  958  196  825 1726 3425 1806]
[1965 3519  247 1411 1034 1791 1586 3592 4977   15 3385 1087 4396 5165
 3628  144  214 3804 2367 1386 1869 2350  452 5316 4385 2009  944 4994
 4921 3871 1516 1580 1442  991 3783  683  388 3884]
[ 319 1038  641  579  414  494  935 1149 1255  530  412  879]
[45 51]
1.0
1
[]
[]
[3149 2972  709 2996 1044  899 2680 2860]
[2820 1145 5485 1310  705 2385 2270 4506 5353  383  433  808  592  147
    4 3225 5176 5448 4107 3819 1016 2805 4477 4356 4739 4250  473 4983
 2762 3948 5325 5191 5199 4755 3811 1790 3730 3641]
[ 268  577  481 1500 1505 1446  301  815  623  201  724  987]
[44 47]
1.0
2
[]
[]
[3183 3187 3289   93  930 3198 2002 1087]
[1260 1954 4878 1097 4873 1182  705 5474  305  499 4474 4023 3313 2774
 4198 2051 2789  696 1379  380 5461 5092 5235 3572 4650  844 3337 5471
 3404 1205 3422  250 2756 2871 5091 4110  341 1471]
[ 695  658  387  945   76  143  573 1134  911  457 1327  349]
[42 23]
1.0
3
[]
[]
[1646 1990  877 2339  557 2500  469 13

In [146]:
cd40_subset_adata = og_adata[:, og_adata.var.index.isin(cd40_filtered_neighbors['cd40_f_in'])]
non_cd40_subset_adata = og_adata[:, ~og_adata.var.index.isin(cd40_filtered_neighbors['cd40_f_in'])]
intervals = [(1e0, 1e1), (1e1, 1e2), (1e2, 1e3), (1e3, 1e4), (1e4, 1e5), (1e5, 1e6)]
y = og_adata.obs['biotin_raw']
perturbed_genes = {}
for i in range(10):
    print(i)
    adata = og_adata.copy()
    perturbed_genes[i] = {}
    for interval in intervals:
        int_non_cd40_subset_adata = non_cd40_subset_adata[:, non_cd40_subset_adata.var['n_counts'].between(interval[0], interval[1])].copy()
        int_cd40_subset_adata = cd40_subset_adata[:, cd40_subset_adata.var['n_counts'].between(interval[0], interval[1])].copy()
        
        total_non_cd40_genes = int_non_cd40_subset_adata.shape[1]
        target_gene_count = int_cd40_subset_adata.shape[1]

        np.random.seed(i)
        # Randomly sample 173 gene indices from the total 318 genes
        selected_genes = np.random.choice(total_non_cd40_genes, target_gene_count, replace=False)
        print(selected_genes)

        # Subset the AnnData object to the selected genes
        int_non_cd40_subset_adata = int_non_cd40_subset_adata[:, selected_genes].copy()
        perturbed_genes[i][interval] = int_non_cd40_subset_adata.var.index

0
[]
[]
[ 989  463  958  196  825 1726 3425 1806]
[1965 3519  247 1411 1034 1791 1586 3592 4977   15 3385 1087 4396 5165
 3628  144  214 3804 2367 1386 1869 2350  452 5316 4385 2009  944 4994
 4921 3871 1516 1580 1442  991 3783  683  388 3884]
[ 319 1038  641  579  414  494  935 1149 1255  530  412  879]
[45 51]
1
[]
[]
[3149 2972  709 2996 1044  899 2680 2860]
[2820 1145 5485 1310  705 2385 2270 4506 5353  383  433  808  592  147
    4 3225 5176 5448 4107 3819 1016 2805 4477 4356 4739 4250  473 4983
 2762 3948 5325 5191 5199 4755 3811 1790 3730 3641]
[ 268  577  481 1500 1505 1446  301  815  623  201  724  987]
[44 47]
2
[]
[]
[3183 3187 3289   93  930 3198 2002 1087]
[1260 1954 4878 1097 4873 1182  705 5474  305  499 4474 4023 3313 2774
 4198 2051 2789  696 1379  380 5461 5092 5235 3572 4650  844 3337 5471
 3404 1205 3422  250 2756 2871 5091 4110  341 1471]
[ 695  658  387  945   76  143  573 1134  911  457 1327  349]
[42 23]
3
[]
[]
[1646 1990  877 2339  557 2500  469 1348]
[3072 37

In [147]:
perturbed_genes

{0: {(1.0, 10.0): Index([], dtype='object'),
  (10.0, 100.0): Index([], dtype='object'),
  (100.0,
   1000.0): Index(['Pigo', 'Inafm2', 'Lpar1', 'Stk11ip', 'Med18', 'Rufy2', 'Tmem241',
         'Adgb'],
        dtype='object'),
  (1000.0,
   10000.0): Index(['Med21', 'Map2k5', 'Aff3', 'Kazn', 'Ints12', 'N4bp2', 'Sh3bp2', 'Itga9',
         'Alkbh7', 'Adss', 'Idh3a', 'Rap1gds1', 'Mgat2', 'Fgd2', 'Ccdc82',
         'Als2', 'Rnpep', 'Gaa', 'Ppp1r14a', 'Sf3a3', 'Foxk1', 'Rsf1', 'Arfgap2',
         'Zfp516', 'Klhl28', 'Nagk', 'Etfdh', 'Vav1', 'Mcfd2', 'Pcyt2', 'Atp8a1',
         'Fip1l1', 'Topors', 'Pgrmc2', 'Akap10', 'Stom', 'Med27', 'Aspscr1'],
        dtype='object'),
  (10000.0,
   100000.0): Index(['S100a4', 'Vdac1', 'Eif3k', 'Gfpt1', 'Clta', 'Fosl2', 'Sema7a',
         'Serpinb9', 'Cox6c', 'Ndufa4', 'Tln1', 'Spcs1'],
        dtype='object'),
  (100000.0, 1000000.0): Index(['Rpl6', 'Rpl27a'], dtype='object')},
 1: {(1.0, 10.0): Index([], dtype='object'),
  (10.0, 100.0): Index([], dtype

In [161]:
aggregated_perturbed_genes = {}
for i in perturbed_genes:
    mygenes = []
    for bins in perturbed_genes[i]:
        for item in perturbed_genes[i][bins]:
            mygenes.append(item)
    aggregated_perturbed_genes['noncd40_' + str(i)] = list(set(mygenes))

In [162]:
with open('../pathway_computations/noncd40_perturbed_genes.pickle', 'wb') as handle:
    pickle.dump(aggregated_perturbed_genes, handle, protocol=4)

In [85]:
for interval in intervals:
    further_subset_adata = subset_adata[:, subset_adata.var['n_counts'].between(interval[0], interval[1])].copy()
    print(further_subset_adata.shape)

(10346, 0)
(10346, 0)
(10346, 8)
(10346, 38)
(10346, 12)
(10346, 2)
